import packages and set image size

In [7]:
import json
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['figure.figsize'] = [10,10]

this funtion reads results from 1 specific run for N iterations

In [10]:
def read_experiment_results(full_path, iterations):
    dir_name = full_path
    results = []
    for it in range(iterations):
        with open(dir_name + 'iteration_' + str(it) + '.json') as json_file:
            if it % 10 == 0:
                print(it)
            data = json.load(json_file)
            results.append(data)
    print("Finished reading ", full_path)
    return results


In this cell you choose which run and which full_path to use to read the results of that 

In [3]:
run_number = [1:15]
iterations = 100
os_string = "osx"
results = []
if os_string == "osx":
    path = "/Users/soren/Work/autolr/dumps/frace/run_"
elif os_string == "w10":
    path = "G:\\My Drive\\results\\autolr-2022-05-25\\run_"
for i in run_number: 
    results.append(read_experiment_results(path + i + '/', iterations))


0
10
20
30
40
Finished reading  /Users/soren/Work/autolr/dumps/frace/run_3/


In [4]:
best = 1
it = 0
best_indivs = {}
averages = []
best_indiv = None
for generation in results[0]:
    new_best = False 
    total_fits = 0
    for indiv in generation:
        total_fits += indiv["fitness"]
        if indiv["fitness"] < best:
            best = indiv["fitness"]
            best_indiv = indiv
            best_indivs[it] = indiv
            new_best = True
    averages.append(total_fits/len(generation))
    if new_best:
        print(f"[{it}] New Best: {best} {best_indiv['smart_phenotype']}")
    it += 1
print(best_indiv)


[0] New Best: -0.6926474545563448 square(sqrt(add(alpha, divide_no_nan(pow(add(alpha, pow(constant(1.56514861e-02), add(divide_no_nan(sqrt(sqrt(grad)), divide_no_nan(add(constant(2.11963334e-01), square(alpha)), constant(2.47663801e-01))), subtract(add(beta, pow(add(beta, add(grad, alpha)), grad)), constant(9.25893264e-01))))), beta), alpha))))
[9] New Best: -0.7871503959449806 subtract(constant(2.28478855e-04), alpha)
{'genotype': [[0], [1], [], [0], [0], [8], [1], [], [0], [1], [], [0, 0, 1, 0, 1, 1], [4, 8, 4], [0, 0, 0], [0, 3, 3], [77], [0, 1, 1], [1], [1, 2], []], 'fitness': -0.7871503959449806, 'tree_depth': 9, 'operation': 'copy', 'id': 157, 'evaluations': [], 'phenotype': 'alpha_func, beta_func, sigma_func, grad_func = lambda shape,  alpha, grad: tf.constant(2.28478855e-04, shape=shape, dtype=tf.float32), lambda shape,  alpha, beta, grad: alpha, lambda shape,  alpha, beta, sigma, grad: tf.math.pow(tf.math.sqrt(tf.constant(9.96148968e-01, shape=shape, dtype=tf.float32)), tf.mat

In [15]:
import re
def get_constants_and_probe(phenotype):
    probe = []
    constant_strings = []
    for tf_constant in re.findall("tf.constant\([0-9]\.[0-9]+e.0[0-9], shape=shape, dtype=tf.float32\)", phenotype):
        value = re.findall("[0-9]\.[0-9]+e.0[0-9]", tf_constant)
        constant_strings.append(tf_constant)
        probe.append(float(value[0]))
    return constant_strings, probe

from bayes_opt import BayesianOptimization
def create_evaluate_optimizer_function(phenotype, params):
    def evaluate_optimizer(**kwargs):
        for key, value in kwargs.items():
            phenotype.replace(key, f"tf.constant({value}, shape=shape, dtype=tf.float32)")
        from evaluators.adaptive_optimizer_evaluator_f_race import train_model
        fitness, other_info = train_model((phenotype, params)) 
        return fitness
    return evaluate_optimizer


def tune_optimizer(n_iter, init_points, phenotype, params):
    constants, probes = get_constants_and_probe(phenotype)
    f = create_evaluate_optimizer_function(phenotype, params)
    pbounds = {}
    params = {}
    i = 0
    for constant, probe_value in zip(constants, probes):
        param_key = 'param_' + str(i)
        pbounds[param_key] = (0, 1)
        params[param_key] = probe_value
        phenotype.replace(constant, param_key, 1)
        i += 1

    optimizer = BayesianOptimization(
        f=f,
        pbounds=pbounds,
        verbose=1
    )

    optimizer.probe(params=params
    )
    
    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
    )

In [16]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
params = {
    "parameters": "parameters/adaptive_autolr_mutation_level_fmni.yml", 
    "popsize": 100, 
    "generations": 100, 
    "elitism": 1, 
    "prob_crossover": 0.0, 
    "prob_mutation": {
        "0": 0.0, 
        "1": 0.01, 
        "2": 0.01, 
        "3": 0.01, 
        "4": 0.05, 
        "5": 0.15, 
        "6": 0.01, 
        "7": 0.01, 
        "8": 0.01, 
        "9": 0.05, "10": 0.15, "11": 0.01, "12": 0.01, "13": 0.01, "14": 0.05, "15": 0.15, "16": 0.01, "17": 0.01, "18": 0.05, "19": 0.15}, "tsize": 2, "grammar": "grammars/adaptive_autolr_grammar_mutate_level.txt", "experiment_name": "dumps/fmni", "run": 1, "save_step": 1, "min_tree_depth": 6, "max_tree_depth": 17, "model": "models/mnist_model.h5", "validation_size": 3500, "fitness_size": 50000, "batch_size": 1000, "epochs": 100, "patience": 5, "fitness_floor": 0, }

tune_optimizer(90, 10, best_indiv['phenotype'], params)

KeyboardInterrupt: 

We can see that best fitness and average fitness improve together until the best individual in generation 124.
After this generation, average fitness descreases, despite the improvements of the best individual.
Let's take a look at the best individuals, why is this happening?

In [ ]:
print(f"[44] {best_indivs[44]['smart_phenotype']} {averages[85]}")
print(f"[86] {best_indivs[86]['smart_phenotype']} {averages[87]}")
print(f"[88] {best_indivs[88]['smart_phenotype']} {averages[97]}")
print(f"[98] {best_indivs[98]['smart_phenotype']} {averages[102]}")
print(f"[103] {best_indivs[103]['smart_phenotype']} {averages[111]}")
print(f"[112] {best_indivs[112]['smart_phenotype']} {averages[123]}")
print(f"[124] {best_indivs[124]['smart_phenotype']} {averages[139]}")
print(f"[140] {best_indivs[140]['smart_phenotype']} {averages[238]}")
print(f"[239] {best_indivs[239]['smart_phenotype']} {averages[493]}")

In [ ]:
ax = plt.axes()
ax.set_facecolor(color="#eff2f1")
ax.spines['bottom'].set_color('#08415c')
ax.spines['top'].set_color('#08415c')
ax.spines['left'].set_color('#08415c')
ax.spines['right'].set_color('#08415c')
ax.xaxis.label.set_color('#08415c')
ax.yaxis.label.set_color('#08415c')
ax.tick_params(axis='x', colors='#08415c')
ax.tick_params(axis='y', colors="#08415c")
plt.plot([x for x in range(len(averages[:250]))], averages[:250], label='population average', color="#7796cb")
for x in best_indivs:
    plt.axvline(x, color='red', linestyle='--')

In [ ]:
from smart_phenotype import readable_phenotype
print(readable_phenotype(best_indivs[44]['phenotype']))
print("---")
print(readable_phenotype(best_indivs[86]['phenotype']))

Momentum term and learning rate are the same values. Mutations in this value are likely very destructive